# Riemann scheme. 

<span style="color:#78AE7E">



<span>

Lets consider same setup as in [ex_3a](https://github.com/AST-Course/AST5110/blob/main/ex_3a.ipynb) Burgers’ equation, i.e.,

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = 0   \tag{1}$$ 

for the domain $x \in (x_0, x_f)$ with $x_0 = −1.4$, $x_f = 2.0$ with initial condition:

$$u(x,t=0) = A\left[\tanh\left(\frac{x+x_c}{W}\right)-\tanh\left(\frac{x-x_c}{W}\right)\right]   \tag{2}$$

whereby $A = 0.02$ , $x_c = 0.70$, $W = 0.1$. Let the solution evolve until time $t_f = 100$. However, let's now implement a new time-step method. Find the conservative form of the equation and implement a Rieman solver [wiki:Rieman Solvers](https://github.com/AST-Course/AST5110/wiki/Riemann-solvers). 

Add this to your library and solve the previous simulation imposing the CFL condition. For this exercise, fill in `nm_lib` the function `evolv_Rie_uadv_burgers`. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')
import matplotlib as mpl
mpl.rc('lines', linewidth=2)

from nm_lib import nm_lib as nm

def u_0(x: np.ndarray, t: float = 0) -> np.ndarray:
    r"""
    Initial condition for the advection equation.

    Parameters
    ----------
    x : `array`
        the x-axis.
    t : `float`
        the time.
    
    Returns
    -------
    `array`
        the initial condition.
    """
    return A * ( np.tanh((x + x_c) / W) - np.tanh((x - x_c) / W) )

In [ ]:
# Setup for animations

from matplotlib import animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [ ]:
A = 0.02 
W = 0.1
x_c = 0.7

tf = 100

#### 1 step

Compute left and right of $u$, i.e., $u_L$ and $u_R$. Note that $u_R = u_{i+1}$ and $u_L = u_i$

#### 2 step
Rewrite the eq 1 in a conservative form to get the Flux. Compute from the left and right variable ($u_L$ and $u_R$) the corresponding flux, i.e., $F_L$ and $F_R$

#### 3 step
Compute the propagating speed. This will be the derivatie of the flux with respect to variable $u$ in absolute values and select the largest between the left and right shells ($v_a$).

#### 4 step
Compute the interface fluxes (Rusanov)

$F^*_{i+1/2}=\frac{F_R+F_L}{2} - \frac{v_a}{2} (U_R -U_L)$

Note that $F^*_{i+1/2}$ is half grid shifted

#### 5 step
Advance in time $u^{n+1}=u^n - dt\times (F^*_{i+1/2}-F^*_{i-1/2})$. 

What is dt taking into account the phase speed ($v_a$)? 


In [ ]:
# Define the domain 
x0 = -1.4
xf = 2.0

nx = 128
nt = 200
xx = np.linspace(x0, xf, nx) 

t, unnt_Lax = nm.evolv_Lax_uadv_burgers(xx, u_0(xx), nt, cfl_cut=0.98, bnd_limits=[1, 0])
t, unnt_Rie = nm.evolv_Rie_uadv_burgers(xx, u_0(xx), nt, bnd_limits=[1, 0])

# Explain the boundary limits

def init(): 
    """Initialize animation"""
    axes.plot(xx,unnt_Rie[:,0], label='Rie')
    axes.plot(xx,unnt_Lax[:,0], label='Lax', ls='--')
    axes.legend(loc='upper right')

def animate(i):
    """Animate the solution with dynamic axis limits"""
    axes.clear()
    axes.plot(xx,unnt_Rie[:,i], label='Rie')
    axes.plot(xx,unnt_Lax[:,i], label='Lax', ls='--')
    axes.set_title('t=%.2f'%t[i])
    axes.legend(loc='upper right')
    axes.set_ylim(-0.005, 0.045)

fig, axes = plt.subplots(figsize=(7, 3))
anim = FuncAnimation(fig, animate, interval=20, frames=nt, init_func=init)
plt.close(); HTML(anim.to_jshtml())

Is this method less or more diffusive than Lax method? Where and when is better or worse than the Lax method? 

<span style="color:#78AE7E">

We see from the animation above that the Riemann scheme is about equally as diffusive as the Lax method. However, the Riemann method more accurately represents the "shock" in the solution, as we can se from the straigh angles, whereas the Lax method stays more "round", even at the edge of the travelling shock. 

<span>

The different approximations of Rieman solvers uses different ways to estimate $F^*$ and phase velocity ($v_a$). Otherwise they do the same steps. 

<span style="color:#78AE7E">

Disregard the cells below!

<span>

---
## TVD scheme. 

Combine the Lax method with the Rieman solver using a flux limiter scheme. For this, one needs to identify how large are the gradients. Lets define: 

$$r^{+}_i = \frac{u_i-u_{i-1}}{u_{i+1}+u_i}$$

What kind of properties do you see on $r^{+}_i$? e.g., what happens when $r\ge0$? or $r\le0$? 

now lets consider the following flux limiter: 

$$\phi = max\left(0,min\left(\theta r,\frac{1+r}{2},\theta \right)\right),\, \theta =[1,2]$$

Finally, combine the Lax from previous excersice with the Riemann solver using the flux limiter as follows: 

$$u^{n+1}_i = u^n_i + dt (f^n_{i+1/2}-f^n_{i-1/2})$$

Where 

$$f^n_{i+1/2} = f^{Riemann}_{i+1/2} + \phi^n_i (f^{L}_{i+1/2}-f^{Riemann}_{i+1/2})$$



In [ ]:
# Define the domain 
x0 = -1.4
xf = 2.0

nx = 128
nt = 200
xx = np.linspace(x0, xf, nx) 

t, unnt = nm.evolve_Lax_Rie_uadv_burgers(xx, u_0(xx), nt, cfl_cut=0.98, ddx=nm.deriv_upw, bnd_limits=[1, 0])

def init(): 
    """Initialize animation"""
    axes.plot(xx,unnt[:,0])
    # axes.plot(xx,unnt_Lax[:,0], label='Lax', ls='--')
    # axes.legend(loc='upper right')

def animate(i):
    """Animate the solution with dynamic axis limits"""
    axes.clear()
    axes.plot(xx,unnt[:,i])
    # axes.plot(xx,unnt_Lax[:,i], label='Lax', ls='--')
    axes.set_title('t=%.2f'%t[i])
    # axes.legend(loc='upper right')
    axes.set_ylim(-0.005, 0.045)

fig, axes = plt.subplots(figsize=(7, 3))
anim = FuncAnimation(fig, animate, interval=20, frames=nt, init_func=init)
plt.close(); HTML(anim.to_jshtml())

When becomes purely Rieman solver? And Lax? Which others methods you can find depending on what is $r$ of $\phi$. 

What would be the corrective numerical flux in order to be a flux-limited?